In [8]:
import pandas as pd
import numpy as np
import ast

In [4]:
raw_data_folder = "../data/ratings.dat"

In [5]:
ratings = pd.read_csv(raw_data_folder, names=["u","v","r"], nrows=1000)

In [6]:
train_data_path =  "../data/matches_train_for_d2v.dat"
test_data_path =  "../data/matches_test.dat"

In [12]:
def load_d2v_formated_data(data_path):
    df = pd.read_csv(data_path)
    df["rated"] = df["rated"].map(ast.literal_eval)
    return df
df = load_d2v_formated_data(train_data_path) # rater, rated, r, m

In [28]:
df = df.loc[:10]
df["idx"] = df.reset_index()["index"].apply(lambda x: "a"+str(x))
df.set_index("idx").loc["a0"]

rated    [971, 1616, 4633, 8305, 9345, 9729, 10148, 132...
Name: a0, dtype: object

In [183]:
from gensim.models import Word2Vec
model = Word2Vec(sentences=df["rated"].values, size=100, window=5, min_count=1, workers=4)

In [184]:
model.train_count

1

In [199]:
model = Word2Vec(size=100, window=5, min_count=1, workers=4)
if model.train_count == 0:
    model.build_vocab(df["rated"].values)
model.train(df["rated"].values, total_examples=model.corpus_count)

TypeError: unhashable type: 'list'

In [196]:
df["rated"].values[0]

['971',
 '1616',
 '4633',
 '8305',
 '9345',
 '9729',
 '10148',
 '13210',
 '14604',
 '14985',
 '18032',
 '18287',
 '18345',
 '19006',
 '19727',
 '21256',
 '22319',
 '22915',
 '24294',
 '24683',
 '26084',
 '33960',
 '34406',
 '35647',
 '37773',
 '38356',
 '39414',
 '40377',
 '43836',
 '47314',
 '49466',
 '53933',
 '54929',
 '61157',
 '63063',
 '64257',
 '65602',
 '69882',
 '73281',
 '73611',
 '75050',
 '75243',
 '83622',
 '87068',
 '88179',
 '88886',
 '91102',
 '93105',
 '97655',
 '98556',
 '105720',
 '113916',
 '116926',
 '120342',
 '121859',
 '127833',
 '130970',
 '131669',
 '138830',
 '140246',
 '144218']

In [38]:
import sys
sys.path.insert(0, "../")
from d2v_recommender import *

In [71]:
df["array"] = df["rated"].apply(lambda x: np.array([1,2,3.5,4.7,17,17]))
df_ = df.copy()
df_ = pd.concat([df_,df_])
df_ = df_.groupby("idx")["array"].apply(np.sum)
df_ = df.iloc[:5]

In [72]:
df_

,rated,idx,array
0,"[971, 1616, 4633, 8305, 9345, 9729, 10148, 132...",a0,"[1.0, 2.0, 3.5, 4.7, 17.0, 17.0]"
1,"[133, 4672, 5577, 7006, 12038, 12243, 17204, 2...",a1,"[1.0, 2.0, 3.5, 4.7, 17.0, 17.0]"
2,"[25083, 93891, 105720, 130815]",a2,"[1.0, 2.0, 3.5, 4.7, 17.0, 17.0]"
3,"[6499, 7964, 11878, 12513, 16357, 18534, 27357...",a3,"[1.0, 2.0, 3.5, 4.7, 17.0, 17.0]"
4,"[1339, 3015, 3132, 12107, 17811, 18074, 18227,...",a4,"[1.0, 2.0, 3.5, 4.7, 17.0, 17.0]"


In [80]:
np.save("local_test.npy",arr=df_.values)
arr=np.load("local_test.npy", allow_pickle=True)
arr

array([[list(['971', '1616', '4633', '8305', '9345', '9729', '10148', '13210', '14604', '14985', '18032', '18287', '18345', '19006', '19727', '21256', '22319', '22915', '24294', '24683', '26084', '33960', '34406', '35647', '37773', '38356', '39414', '40377', '43836', '47314', '49466', '53933', '54929', '61157', '63063', '64257', '65602', '69882', '73281', '73611', '75050', '75243', '83622', '87068', '88179', '88886', '91102', '93105', '97655', '98556', '105720', '113916', '116926', '120342', '121859', '127833', '130970', '131669', '138830', '140246', '144218']),
        'a0', array([ 1. ,  2. ,  3.5,  4.7, 17. , 17. ])],
       [list(['133', '4672', '5577', '7006', '12038', '12243', '17204', '25083', '31847', '35647', '38594', '40377', '43970', '55238', '56889', '57473', '62064', '70118', '78864', '79507', '82946', '85264', '105269', '117149', '119447', '121859', '134835', '136180', '144218', '146508', '151118']),
        'a1', array([ 1. ,  2. ,  3.5,  4.7, 17. , 17. ])],
       [list

In [62]:
def makeArray(text):
    return np.fromstring(text,sep=' ')

# pd.read_csv("../data/dev/models/rater.vectors").loc[0][1]
np.fromstring('[ 1.37519860e-03  2.68070668e-04 -2.25159994e-04 -7.21103221e-04\n -2.76153674e-04 -1.11587590e-03 -1.06375094e-03 -1.35301123e-03\n  1.19839667e-03 -6.18751568e-04 -5.82824345e-04 -4.24984319e-04\n -1.24673557e-03  3.28346723e-05  3.11861404e-05  7.53764325e-05\n -8.65640031e-05  1.76705187e-04 -6.43181498e-04  9.94793605e-04\n -1.05464924e-03  1.74464134e-04 -1.09542906e-03 -1.75682851e-03\n -6.18919148e-05 -3.15550977e-04 -1.92337448e-03  7.73766253e-04\n -5.85743634e-04 -3.25974106e-04 -9.44319763e-05 -8.16533109e-04\n -8.34610255e-04 -4.90881270e-04 -1.20761525e-03 -6.04122411e-04\n -6.59026788e-04 -9.21453466e-04 -3.69178859e-04  5.56204293e-04\n  5.18006913e-04  4.40018630e-05 -1.86226913e-04  1.63235905e-04\n  1.18702235e-04  2.79508909e-04 -4.05405444e-04  3.97378521e-04\n -6.38739089e-04  1.50395106e-04]',
             sep=" ")

C:\Users\charl\miniconda3\envs\d2v_env\lib\site-packages\ipykernel_launcher.py:6: DeprecationWarning: string or file could not be read to its end due to unmatched data; this will raise a ValueError in the future.
  


array([], dtype=float64)

In [2]:
def load_d2v_formated_data(data_path):
    df = pd.read_csv(data_path)
    df["rated"] = df["rated"].map(ast.literal_eval)
    return df
df = load_d2v_formated_data(train_data_path)

NameError: name 'train_data_path' is not defined

In [203]:
def list_shuffler(x):
    np.random.shuffle(x)
    return x
df["rated"].apply(list_shuffler)

0      [971, 35647, 87068, 13210, 9345, 127833, 14024...
1      [5577, 12243, 25083, 55238, 7006, 134835, 6206...
2                         [25083, 130815, 93891, 105720]
3      [27357, 7964, 6499, 136297, 11878, 174211, 132...
4      [102366, 3015, 12107, 91366, 135266, 87789, 25...
                             ...                        
995    [78597, 85704, 33216, 67265, 8196, 13029, 5505...
996    [25083, 147908, 91366, 59874, 115192, 12795, 7...
997          [102751, 3446, 12845, 53583, 71636, 112075]
998    [115129, 135266, 12227, 24734, 163591, 26193, ...
999    [68140, 105522, 115872, 2111, 70720, 45418, 31...
Name: rated, Length: 1000, dtype: object

In [204]:
df["rated"]

0      [971, 35647, 87068, 13210, 9345, 127833, 14024...
1      [5577, 12243, 25083, 55238, 7006, 134835, 6206...
2                         [25083, 130815, 93891, 105720]
3      [27357, 7964, 6499, 136297, 11878, 174211, 132...
4      [102366, 3015, 12107, 91366, 135266, 87789, 25...
                             ...                        
995    [78597, 85704, 33216, 67265, 8196, 13029, 5505...
996    [25083, 147908, 91366, 59874, 115192, 12795, 7...
997          [102751, 3446, 12845, 53583, 71636, 112075]
998    [115129, 135266, 12227, 24734, 163591, 26193, ...
999    [68140, 105522, 115872, 2111, 70720, 45418, 31...
Name: rated, Length: 1000, dtype: object